#Converting XML annotations to CSV format

This is done so that the data can be inputted into the RetinaNet model.

In [0]:
from bs4 import BeautifulSoup
import pandas as pd

In [9]:
from google.colab import drive
drive.mount("/content/drive/")

#Computer/Dad's iMac (My User)/Desktop/Project/Signet_ring_cell_dataset-20190621T091454Z-001/sig-train-pos/2018_64982_1-3_2019-02-25 21_57_36-lv0-33516-59515-2003-2010.xml

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
xml_files = pd.read_csv('/content/drive/My Drive/QMUL/Project/Signet_ring_cell_dataset-20190621T091454Z-001/xml_list.csv')

In [11]:
xml_files.head()

,filename
0,2018_64982_1-3_2019-02-25 21_57_36-lv0-50380-5...
1,2018_64982_1-3_2019-02-25 21_57_36-lv0-33516-5...
2,2018_64982_1-3_2019-02-25 21_57_36-lv0-34589-6...
3,2018_64982_1-3_2019-02-25 21_57_36-lv0-36515-5...
4,2018_64982_1-3_2019-02-25 21_57_36-lv0-37528-6...


In [0]:
jpeg_files = pd.read_csv('/content/drive/My Drive/QMUL/Project/Signet_ring_cell_dataset-20190621T091454Z-001/jpeg_list.csv')

In [13]:
jpeg_files.head()

,filename
0,2018_64982_1-3_2019-02-25 21_57_36-lv0-33516-5...
1,2018_64982_1-3_2019-02-25 21_57_36-lv0-34589-6...
2,2018_64982_1-3_2019-02-25 21_57_36-lv0-36515-5...
3,2018_64982_1-3_2019-02-25 21_57_36-lv0-37528-6...
4,2018_64982_1-3_2019-02-25 21_57_36-lv0-38368-6...


In [0]:
from bs4 import BeautifulSoup
import pandas as pd

class xml_to_csv:

	def __init__(self, path_to_file, tags_to_extract, to_csv = False):
		""" names_to_extract MUST be a list or tuple + might need to have equal number of entries in file"""

		self.path = path_to_file
		self.to_csv = to_csv
		self.soup = BeautifulSoup(open(path_to_file, 'r'))
		file_name_extract = self.soup.findAll('filename')
		self.file_name = [t.text for t in file_name_extract][0]

		self.titles = tags_to_extract

	def extract(self):

		empty = []

		for el in self.titles:
			col = self.soup.findAll(el)
			col_item = [t.text for t in col]
			empty.append({el: col_item})

		data_out = pd.DataFrame()
		for el in empty:
			dframe = pd.DataFrame(el)
			header = list(dframe.columns)[0]
			data_out[f'{header}'] = dframe[f'{header}']

		data_out['filename'] = self.file_name

		if self.to_csv:
			data_out.to_csv(f'{self.file_name} output.csv', index=False)
		return data_out


if __name__ == '__main__':
	for file in xml_files['filename']:
		converter = xml_to_csv(path_to_file='/content/drive/My Drive/QMUL/Project/Signet_ring_cell_dataset-20190621T091454Z-001/sig-train-pos/{}'.format(file),
                         tags_to_extract = ('xmin', 'ymin', 'xmax' ,'ymax','name'), to_csv=True)
		df = converter.extract()
# if __name__ == '__main__':
# 	converter = xml_to_csv(path_to_file='/content/drive/My Drive/QMUL/Project/Signet_ring_cell_dataset-20190621T091454Z-001/sig-train-pos/2018_64982_1-3_2019-02-25 21_57_36-lv0-33516-59515-2003-2010.xml',
#                         tags_to_extract = ('xmin', 'ymin', 'xmax' ,'ymax','name'), to_csv=True)
# 	df = converter.extract()


In [0]:
all_dataframes = []

# get data file names
for file in jpeg_files['filename']:
  all_dataframes.append(pd.read_csv('{} output.csv'.format(file)))

# Concatenate all WSI annotations into one DataFrame
WSI_df = pd.concat(all_dataframes, ignore_index=True)

In [0]:
# Reorder columns and rename columns
WSI_df = WSI_df[['filename', 'xmin', 'ymin', 'xmax', 'ymax','name']]
WSI_df = WSI_df.rename(columns={"filename": "image_name", "xmin": "x_min", "ymin": "y_min", 
                       "xmax": "x_max", "ymax": "y_max", "name": "class_name"})

In [38]:
WSI_df

,image_name,x_min,y_min,x_max,y_max,class_name
0,2018_64982_1-3_2019-02-25 21_57_36-lv0-33516-5...,1346,94,1413,189,ring_cell_cancer
1,2018_64982_1-3_2019-02-25 21_57_36-lv0-33516-5...,1205,165,1302,236,ring_cell_cancer
2,2018_64982_1-3_2019-02-25 21_57_36-lv0-33516-5...,1214,231,1280,318,ring_cell_cancer
3,2018_64982_1-3_2019-02-25 21_57_36-lv0-33516-5...,1410,346,1503,430,ring_cell_cancer
4,2018_64982_1-3_2019-02-25 21_57_36-lv0-33516-5...,1366,291,1452,373,ring_cell_cancer
...,...,...,...,...,...,...
9705,G1901976_2019-04-30 14_03_43-lv0-42750-26583-2...,1656,321,1717,373,ring_cell_cancer
9706,G1901976_2019-04-30 14_03_43-lv0-42750-26583-2...,1441,367,1484,445,ring_cell_cancer
9707,G1901976_2019-04-30 14_03_43-lv0-42750-26583-2...,1382,816,1449,857,ring_cell_cancer
9708,G1901976_2019-04-30 14_03_43-lv0-42750-26583-2...,1851,998,1924,1039,ring_cell_cancer


In [0]:
WSI_df.to_csv('WSI_Annotations.csv', index = False)